In [1]:
import sys
sys.setrecursionlimit(15000)

import torch
import torch.nn.functional as F
from torch import nn
import numpy as np
from torch.autograd import Variable
from torchvision.datasets.mnist import MNIST
device='cuda:4'
from torchvision import datasets, transforms
import pandas as pd
import numpy as np
from path import Path
import torch.utils.data as data
from imageio import imread
import os
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F 
import matplotlib.pyplot as plt
%matplotlib inline 
from torch.utils.data.sampler import SubsetRandomSampler
from PIL import *
import os
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split

In [2]:
seed=4
df=pd.read_csv("../../Multi-modality/Model/Prostrate/uni model/MLP/preprocessed_prostrate.csv")
no_of_rows,no_of_cols=df.shape
print (no_of_rows,no_of_cols)
df.head()

2424 106


,id_ref,gen_name,v1,v2,v3,v4,v5,v6,v7,v8,...,v95,v96,v97,v98,v99,v100,v101,v102,v103,v104
0,Hs6-1-6-1,VHL,-0.009053,0.036120,-0.13310,0.08359,-0.3133,0.09471,-0.19860,0.1129,...,-0.1065,-0.19410,-0.109500,-0.16090,-2.267000e-01,-0.06939,-0.22780,-0.317900,-0.3523,-0.2771
1,Hs6-1-16-1,SMOX,0.000000,0.000000,0.00000,0.11160,0.0000,0.00000,0.00000,0.0000,...,0.0000,0.00000,0.000000,0.17460,-5.180000e-10,0.00000,0.00000,0.000000,-0.3898,-0.1275
2,Hs6-1-6-2,ZNF148,0.189100,-0.175300,0.20240,0.05745,-0.5522,-0.04709,-0.19840,-0.2156,...,0.1284,-0.18470,0.007602,0.02113,4.399000e-01,-0.06516,0.01167,0.304800,0.3409,-0.1266
3,Hs6-1-10-2,MAP4K2,0.335300,0.145700,0.02996,-0.03641,-0.2931,0.23760,-0.01799,0.1953,...,0.3081,-0.25550,0.255800,-0.08655,3.707000e-02,0.03404,0.13550,-0.008265,0.1549,0.4213
4,Hs6-1-15-2,MAPK4,0.310600,0.008775,0.51170,-0.05886,-0.5037,0.36220,0.24620,0.4525,...,0.3401,-0.04517,0.493200,-0.22720,-1.430000e-01,0.26500,0.14160,0.334100,0.3155,0.5046


In [3]:
header_of_pd=['v'+str(i+1) for i in range(no_of_cols-2)]
X=np.array(df[header_of_pd])
print(X.shape)
print((df['gen_name'][0].lower()))

(2424, 104)
vhl


In [9]:
class Net(torch.nn.Module):
    def __init__(self, n_feature,out_features):
        super(Net, self).__init__()
        self.hidden1 = torch.nn.Linear(n_feature,900)
        self.hidden2 = torch.nn.Linear(900,600)
        self.hidden3 = torch.nn.Linear(600,400)
        self.hidden4 = torch.nn.Linear(400,100)
        self.hidden5 =torch.nn.Linear (100,10)
        self.predict = torch.nn.Linear(10,out_features)
        self.bn1 = nn.BatchNorm1d(num_features=900)
        self.bn2 = nn.BatchNorm1d(num_features=600)
        self.bn3 = nn.BatchNorm1d(num_features=400)
        self.bn4 = nn.BatchNorm1d(num_features=100)
        self.bn5 = nn.BatchNorm1d(num_features=10)
    def forward(self, x):
        x = F.relu(self.hidden1(x))
#         x=self.bn1(x)
        x = F.relu(self.hidden2(x))
#         x=self.bn2(x)
        x =F.relu(self.hidden3(x))
#         x=self.bn3(x)
        x =F.relu(self.hidden4(x))
#         x=self.bn4(x)
        #x =F.relu(self.hidden5(x))
#         x=self.bn5(x)
        
        #x=F.relu(self.predict(x))
        return x
MLP_model=Net(104,2)
MLP_model.load_state_dict(torch.load('../../Multi-modality/Model/Prostrate/uni model/MLP/80MLP_prostate.pt'))


In [10]:
MLP_model(torch.rand(1,104)).shape

torch.Size([1, 100])

In [11]:
gene_name_mlp=[]
features_mlp_=[]
for f in range(X.shape[0]):
    print(f)
    gene_name_mlp.append(df['gen_name'][f].lower())
    feature_from_model=MLP_model(torch.tensor(X[f].reshape(1,-1),dtype=torch.float32))
    to_append=feature_from_model[0].data.numpy().tolist()
    features_mlp_.append(to_append)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2089
2090
2091
2092
2093
2094
2095
2096
2097
2098
2099
2100
2101
2102
2103
2104
2105
2106
2107
2108
2109
2110
2111
2112
2113
2114
2115
2116
2117
2118
2119
2120
2121
2122
2123
2124
2125
2126
2127
2128
2129
2130
2131
2132
2133
2134
2135
2136
2137
2138
2139
2140
2141
2142
2143
2144
2145
2146
2147
2148
2149
2150
2151
2152
2153
2154
2155
2156
2157
2158
2159
2160
2161
2162
2163
2164
2165
2166
2167
2168
2169
2170
2171
2172
2173
2174
2175
2176
2177
2178
2179
2180
2181
2182


In [12]:
print(len(features_mlp_))
print(len(gene_name_mlp))

2424
2424


In [13]:
dataframe1 = pd.DataFrame({'gene_name':gene_name_mlp,'feature_MLP':features_mlp_})
tags= dataframe1['feature_MLP'].apply(pd.Series)
tags = tags.rename(columns = lambda x : 'tag_' + str(x))
dataframe1=pd.concat([dataframe1['gene_name'], tags[:]], axis=1)
dataframe1.head()
dataframe1.to_csv('Prostrate_MLP_features_ankit.csv' ,encoding='utf-8', index=False)

In [15]:
q=pd.read_csv('Prostrate_MLP_features_ankit.csv')
q.head()
#MLP FEATURES COMPLETED
# len(q)

,gene_name,tag_0,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8,...,tag_90,tag_91,tag_92,tag_93,tag_94,tag_95,tag_96,tag_97,tag_98,tag_99
0,vhl,0.000000,0.000000,0.0,22.241405,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,10.266189
1,smox,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,7.911898,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,12.720570,0.000000
2,znf148,0.000000,0.000000,0.0,21.678095,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,10.640351
3,map4k2,20.404165,23.852463,0.0,0.000000,43.247234,0.0,31.033705,0.203660,56.684559,...,0.000000,18.108454,0.0,0.0,0.0,28.755461,5.603198,13.697145,63.017044,0.000000
4,mapk4,42.203545,52.076824,0.0,0.000000,89.526337,0.0,67.578827,10.026752,108.900192,...,3.838364,40.393089,0.0,0.0,0.0,57.628872,18.609310,32.895622,120.597061,0.000000


In [18]:
lstm = nn.LSTM(input_size=4,hidden_size=200,bidirectional=True) #input_size hidden_size num_layers 
lstm2=nn.LSTM(input_size=400,hidden_size=100)
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        
        self.fc1=nn.Linear(100, 500)
        
        self.relu=nn.ReLU(inplace=True)
        self.fc2=nn.Linear(500, 300)
        self.fc3=nn.Linear(300,100)
        self.bn1 = nn.BatchNorm1d(num_features=500)
        self.bn2 = nn.BatchNorm1d(num_features=300)
        self.bn3 = nn.BatchNorm1d(num_features=100)

        self.fc4=nn.Linear(100, 2)
        self.sigm=nn.Sigmoid()
        

    def forward(self, x):
        y=self.relu(self.fc3(self.relu(self.fc2(self.relu(self.fc1(x))))))
        return(y)
    


!nvidia-smi
device='cuda:4'
model1=lstm.to(device)
model2=MLP().to(device)
model3=lstm2.to(device)
model1.load_state_dict(torch.load('../../Multi-modality/Model/Prostrate/uni model/BiLSTM/59BiLSTMprostrateL1.pt'))
model2.load_state_dict(torch.load('../../Multi-modality/Model/Prostrate/uni model/BiLSTM/59BiLSTMprostrateL2.pt'))
model3.load_state_dict(torch.load('../../Multi-modality/Model/Prostrate/uni model/BiLSTM/59BiLSTMprostrateL3.pt'))

Wed Jun 12 23:48:27 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.111                Driver Version: 384.111                   |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:04:00.0  On |                  N/A |
|  0%   38C    P2    58W / 250W |   1023MiB / 11172MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:05:00.0 Off |                  N/A |
| 45%   76C    P2   192W / 250W |   5497MiB / 11172MiB |     98%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [19]:
def letterToIndex(letter):
    return all_letters.find(letter)
def letterToTensor(letter):
  tensor=torch.zeros(1,4)
  tensor[0][letterToIndex(letter)] = 1
  return tensor
def lineToTensor(line):
  tensor=torch.zeros(len(line),1,4)
  for li,letters in enumerate(line):
    tensor[li][0][letterToIndex(letters)]=1
  return(tensor)

In [21]:
g=pd.read_csv('../../Multi-modality/Model/Prostrate/uni model/BiLSTM/preprocessed_prostrate.csv')
mapz=pd.read_csv('../../Multi-modality/Model/Prostrate/uni model/BiLSTM/sequence.csv')
key=mapz['gene'].tolist()
sequence=mapz['sequence'].tolist()

dictionary = dict(zip(key, sequence))

In [23]:
all_letters="ACTG"
 

gene_name_lstm=[]
gene_sequence_lstm=[]
gene_lstm_features=[]
for V in (range(len(g['gen_name']))):
    print(V)
    j=g['gen_name'][V]
    GN=j.lower()
    if GN not in dictionary.keys():
        continue
    lin2TenGene=np.array(lineToTensor(dictionary[GN][:70]))
#     print(lin2TenGene.shape) #70 1 4
    gene_name_lstm.append(j.lower())
    gene_sequence_lstm.append(lin2TenGene)
    inp=torch.tensor(lin2TenGene).to(device)

    h0 = torch.zeros(2, 1, 200).to(device)   #(num_layers * num_directions, batch, hidden_size)
    c0 = torch.zeros(2, 1, 200).to(device)
    h_x2=c_x2=torch.zeros(1, 1, 100).to(device)
    output, (hn, cn) = model1(inp, (h0, c0))  
    output,(_,_)=model3(output,(h_x2,c_x2))
    
    q1,q2,q3=output.shape
    out_last=((output[q1-1]))
    After_fc=model2(out_last)
    gene_lstm_features.append(After_fc[0].cpu().data.numpy())
    
            
#             After_fc=model2(out_last)
    
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072


In [24]:
# len(gene_lstm_features)
len(gene_name_lstm)

1763

In [25]:
dataframe2 = pd.DataFrame({'gene_name':gene_name_lstm,'feature_LSTM':gene_lstm_features})
# dataframe2.to_csv('ILD_LSTM_features.csv')
tags= dataframe2['feature_LSTM'].apply(pd.Series)
tags = tags.rename(columns = lambda x : 'tag_' + str(x))
dataframe2=pd.concat([dataframe2['gene_name'], tags[:]], axis=1)
dataframe2.to_csv('prostrate_LSTM_features_ankit.csv' ,encoding='utf-8', index=False)
dataframe2.head()

,gene_name,tag_0,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8,...,tag_90,tag_91,tag_92,tag_93,tag_94,tag_95,tag_96,tag_97,tag_98,tag_99
0,vhl,0.090377,0.0,0.000000,0.798924,0.435122,0.420819,0.0,0.000000,1.809195,...,0.574239,0.185180,0.000000,1.240222,1.611635,2.553779,4.818268,1.098558,0.000000,0.825801
1,smox,0.000000,0.0,0.288466,1.943891,0.000000,0.000000,0.0,0.023711,2.804568,...,1.821619,0.000000,0.000000,0.207053,2.304634,3.305257,5.987690,1.415831,0.000000,0.812028
2,znf148,0.000000,0.0,0.000000,0.373468,0.033001,0.985937,0.0,0.000000,0.340675,...,1.074406,0.787669,0.000000,0.998999,0.611088,1.699283,0.881863,0.000000,0.287414,1.615122
3,map4k2,0.000000,0.0,0.000000,7.312558,0.000000,1.097014,0.0,0.000000,0.000000,...,0.597625,0.000000,1.959956,0.450929,0.638452,3.819741,4.103009,0.000000,0.054212,0.000000
4,mapk4,0.922063,0.0,0.000000,0.868829,0.149503,0.867757,0.0,0.000000,1.014128,...,0.587482,1.080462,0.000000,1.295988,0.889268,0.612330,0.428933,0.028814,0.717688,1.345990


In [26]:
uv=pd.read_csv("prostrate_LSTM_features_ankit.csv")
print(len(uv))

uv.head()
#LSTM FEATURES COMPLETED

1763


,gene_name,tag_0,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8,...,tag_90,tag_91,tag_92,tag_93,tag_94,tag_95,tag_96,tag_97,tag_98,tag_99
0,vhl,0.090377,0.0,0.000000,0.798924,0.435122,0.420819,0.0,0.000000,1.809195,...,0.574239,0.185180,0.000000,1.240222,1.611635,2.553779,4.818268,1.098558,0.000000,0.825801
1,smox,0.000000,0.0,0.288466,1.943891,0.000000,0.000000,0.0,0.023711,2.804568,...,1.821619,0.000000,0.000000,0.207053,2.304634,3.305257,5.987690,1.415831,0.000000,0.812028
2,znf148,0.000000,0.0,0.000000,0.373468,0.033001,0.985937,0.0,0.000000,0.340675,...,1.074406,0.787669,0.000000,0.998999,0.611088,1.699283,0.881863,0.000000,0.287414,1.615122
3,map4k2,0.000000,0.0,0.000000,7.312558,0.000000,1.097014,0.0,0.000000,0.000000,...,0.597625,0.000000,1.959956,0.450929,0.638452,3.819741,4.103009,0.000000,0.054212,0.000000
4,mapk4,0.922063,0.0,0.000000,0.868829,0.149503,0.867757,0.0,0.000000,1.014128,...,0.587482,1.080462,0.000000,1.295988,0.889268,0.612330,0.428933,0.028814,0.717688,1.345990


In [40]:
len(uv)

1763

In [29]:
dfs=pd.read_csv('../../Multi-modality/Model/Prostrate/uni model/CapN/PROSTRATE_GENE_TO_PDB.csv')
final_df=dfs[pd.notnull(dfs['Cross-reference (PDB)'])]
Gene_names=list(final_df['Gene names  (primary )'])
PDB_idz=list(final_df['Cross-reference (PDB)'])
gene_names=[]
for i in range (len(Gene_names)):
    gene_names.append(Gene_names[i].lower())
dictionary = dict(zip(gene_names, PDB_idz))
inp=pd.read_csv('../../Multi-modality/Model/Prostrate/uni model/BiLSTM/preprocessed_prostrate.csv')
prostate_gene_names=list(inp['gen_name'])
prostate_gene_names=[i.lower() for i in prostate_gene_names]
dictionary[prostate_gene_names[0]]

'1LM8;1LQB;1VCB;3ZRC;3ZRF;3ZTC;3ZTD;3ZUN;4AJY;4AWJ;4B95;4B9K;4BKS;4BKT;4W9C;4W9D;4W9E;4W9F;4W9G;4W9H;4W9I;4W9J;4W9K;4W9L;4WQO;5LLI;5N4W;5NVV;5NVW;5NVX;5NVY;5NVZ;5NW0;5NW1;5NW2;5T35;6BVB;6FMI;6FMJ;6FMK;6GFX;6GFY;6GFZ;6GMN;6GMQ;6GMR;6GMX;'

In [30]:
loc_3D=Path('/home1/pratik/Multi-modality/testrun')

In [31]:
def load_as_float(path):
    f = Image.open(path)
    f=f.resize((100,100))
    rgb = f.convert('RGB')
    return(np.array(rgb))

In [32]:
!nvidia-smi

Wed Jun 12 23:58:27 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.111                Driver Version: 384.111                   |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:04:00.0  On |                  N/A |
| 47%   68C    P2   125W / 250W |   6109MiB / 11172MiB |     50%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:05:00.0 Off |                  N/A |
| 49%   83C    P2   224W / 250W |   5497MiB / 11172MiB |     99%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [34]:
NUM_CLASSES = 2
NUM_ROUTING_ITERATIONS = 3
def softmax(input, dim=1):
    transposed_input = input.transpose(dim, len(input.size()) - 1)
    softmaxed_output = F.softmax(transposed_input.contiguous().view(-1, transposed_input.size(-1)), dim=-1)
    return softmaxed_output.view(*transposed_input.size()).transpose(dim, len(input.size()) - 1)

class CapsuleLayer(nn.Module):
    def __init__(self, num_capsules, num_route_nodes, in_channels, out_channels, kernel_size=None, stride=None,
                 num_iterations=NUM_ROUTING_ITERATIONS):
        super(CapsuleLayer, self).__init__()

        self.num_route_nodes = num_route_nodes
        self.num_iterations = num_iterations

        self.num_capsules = num_capsules

        if num_route_nodes != -1:
            self.route_weights = nn.Parameter(torch.randn(num_capsules, num_route_nodes, in_channels, out_channels))
        else:
            self.capsules = nn.ModuleList(
                [nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=0) for _ in
                 range(num_capsules)])

    def squash(self, tensor, dim=-1):
        squared_norm = (tensor ** 2).sum(dim=dim, keepdim=True)
        scale = squared_norm / (1 + squared_norm)
        return scale * tensor / torch.sqrt(squared_norm)

    def forward(self, x):
        if self.num_route_nodes != -1:
#             print("x",x[None, :, :, None, :].shape)
#             print("route",self.route_weights[:, None, :, :, :].shape)
            priors = x[None, :, :, None, :] @ self.route_weights[:, None, :, :, :]
#             print(priors.shape)

            logits = Variable(torch.zeros(*priors.size())).to(device)
#             print("logits-"+str(logits.shape))
            for i in range(self.num_iterations):
                probs = softmax(logits, dim=2)
                outputs = self.squash((probs * priors).sum(dim=2, keepdim=True))
#                 print("onii",outputs.shape)

                if i != self.num_iterations - 1:
                    delta_logits = (priors * outputs).sum(dim=-1, keepdim=True)
#                     print("war",delta_logits.shape)
                    logits = logits + delta_logits
    
        else:
            outputs = [capsule(x).view(x.size(0), -1, 1) for capsule in self.capsules]
#             print("outputs-"+str(outputs[0].shape))
            outputs = torch.cat(outputs, dim=-1)
            outputs = self.squash(outputs)

        return outputs
class CapsuleNet(nn.Module):
    def __init__(self):
        super(CapsuleNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=16, stride=1)
        self.pool=nn.MaxPool2d(3)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=12, stride=1)
        
        self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=9, stride=1)
        self.relu=nn.ReLU()
#         self.dummy_linear=nn.Linear(199712,32*16*16)
        self.primary_capsules = CapsuleLayer(num_capsules=8, num_route_nodes=-1, in_channels=256, out_channels=32,
                                             kernel_size=9, stride=2)
        self.digit_capsules = CapsuleLayer(num_capsules=NUM_CLASSES, num_route_nodes=9248, in_channels=8,
                                           out_channels=16)

        self.decoder = nn.Sequential(
            nn.Linear(16 * NUM_CLASSES, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 784),
            nn.Sigmoid()
        )

    def forward(self, x, y=None):
        x =F.relu(self.conv3((F.relu(self.conv2(self.pool(F.relu(self.conv1(x), inplace=True)))))))
#         print(x.shape)
        x = self.primary_capsules(x)
#         print(x.shape)
        
#         dummy=x.permute(0,2,1)
#         dummy=dummy_linear(dummy)
#         x=dummy.permute(0,2,1)
        x = self.digit_capsules(x).squeeze().transpose(0, 1)



       

        return x

device = 'cpu'
model_3d=CapsuleNet().to(device)
model_3d.load_state_dict(torch.load('../../Multi-modality/Model/Prostrate/uni model/CapN/75percentcapsulenet3d_good.pt',map_location='cpu'))

In [35]:
u=model_3d(torch.rand(1,3,200,200).to(device))
u.shape

torch.Size([16, 2])

In [36]:
u=model_3d(torch.rand(5,3,200,200).to(device))
u.shape

torch.Size([5, 2, 16])

In [37]:
s=0
gene_name_for3d=[]
feature_for_3d=[]
uz=0
for i in prostate_gene_names:
    print(uz)
    uz=uz+1
    a=0
    if i  in dictionary.keys():
        pdb_id_=dictionary[i]
        pdbid_list=pdb_id_.split(';')
        for j in pdbid_list: 
            q=j.lower()
            exists = os.path.isdir(loc_3D/q)
            if exists:
                image_loc=loc_3D/q
                images= sorted((image_loc).files('*.png'))
                if(len(images)>=4):
                    imp=np.zeros((200,200,3))            
                    imp[0:100,0:100,:]=np.array(load_as_float(images[0]))
                    imp[0:100,100:200,:]=np.array(load_as_float(images[1]))
                    imp[100:200,0:100,:]=np.array(load_as_float(images[2]))
                    imp[100:200,100:200,:]=np.array(load_as_float(images[3]))
                    
                    input_for_cp=torch.tensor(imp).permute(2,0,1).unsqueeze_(0).float()
                    
                    output_feature=model_3d(input_for_cp.to(device)).reshape(-1).cpu().data.numpy()
                    
                    gene_name_for3d.append(i)
                    feature_for_3d.append(output_feature)
                    
                    s=s+1
                    break
         

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072


In [38]:
dataframe3 = pd.DataFrame({'gene_name':gene_name_for3d,'feature_3d':feature_for_3d})

tags= dataframe3['feature_3d'].apply(pd.Series)
tags = tags.rename(columns = lambda x : 'tag_' + str(x))
dataframe3=pd.concat([dataframe3['gene_name'], tags[:]], axis=1)
dataframe3.to_csv('Ankitprostate_3d.csv' ,encoding='utf-8', index=False)
dataframe3.head()

,gene_name,tag_0,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8,...,tag_22,tag_23,tag_24,tag_25,tag_26,tag_27,tag_28,tag_29,tag_30,tag_31
0,vhl,-0.004627,0.245317,0.004466,0.069272,-0.003327,-0.263032,-0.003749,0.251758,-0.003560,...,0.003328,-0.229637,0.004711,0.266983,-0.003079,0.228596,0.003680,-0.258568,-0.003935,0.195633
1,ndufa10,0.275199,0.009126,-0.289780,0.004290,0.173321,-0.012266,0.259847,0.010369,0.283457,...,-0.258688,-0.011406,-0.293413,0.009331,0.251243,0.009058,-0.211729,-0.013103,0.264524,0.007954
2,ube4b,0.290513,0.000389,-0.282832,0.000397,0.173428,-0.000026,0.258021,-0.000054,0.284236,...,-0.266636,-0.000551,-0.294954,0.000467,0.244364,0.000293,-0.203645,-0.000146,0.275976,0.000081
3,foxo3,0.281134,0.000155,-0.286166,0.000090,0.173116,-0.000050,0.261762,0.000328,0.288521,...,-0.266013,-0.000130,-0.291289,0.000215,0.253075,0.000154,-0.203884,-0.000096,0.268459,-0.000015
4,rap2a,0.000149,0.245117,-0.000038,0.066403,-0.000030,-0.267965,-0.000021,0.249047,0.000241,...,-0.000095,-0.223775,0.000131,0.271736,-0.000234,0.235593,0.000200,-0.267288,0.000179,0.193028


In [39]:
xx=pd.read_csv('Ankitprostate_3d.csv')
xx

,gene_name,tag_0,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8,...,tag_22,tag_23,tag_24,tag_25,tag_26,tag_27,tag_28,tag_29,tag_30,tag_31
0,vhl,-0.004627,0.245317,0.004466,0.069272,-0.003327,-2.630324e-01,-0.003749,0.251758,-0.003560,...,0.003328,-0.229637,0.004711,0.266983,-3.079333e-03,0.228596,0.003680,-2.585681e-01,-0.003935,0.195633
1,ndufa10,0.275199,0.009126,-0.289780,0.004290,0.173320,-1.226591e-02,0.259847,0.010369,0.283457,...,-0.258688,-0.011406,-0.293413,0.009331,2.512433e-01,0.009058,-0.211729,-1.310255e-02,0.264524,0.007954
2,ube4b,0.290513,0.000389,-0.282832,0.000397,0.173428,-2.565146e-05,0.258021,-0.000054,0.284236,...,-0.266636,-0.000551,-0.294954,0.000467,2.443639e-01,0.000293,-0.203645,-1.463565e-04,0.275976,0.000081
3,foxo3,0.281134,0.000155,-0.286166,0.000090,0.173116,-5.020739e-05,0.261762,0.000328,0.288521,...,-0.266013,-0.000130,-0.291289,0.000215,2.530747e-01,0.000154,-0.203884,-9.553084e-05,0.268459,-0.000015
4,rap2a,0.000149,0.245117,-0.000038,0.066403,-0.000030,-2.679647e-01,-0.000021,0.249047,0.000241,...,-0.000095,-0.223775,0.000131,0.271736,-2.339428e-04,0.235593,0.000200,-2.672885e-01,0.000179,0.193028
5,il33,0.000201,0.248587,-0.000218,0.062095,0.000094,-2.687362e-01,0.000423,0.247374,0.000351,...,-0.000063,-0.231921,-0.000101,0.277009,2.751398e-04,0.226011,-0.000152,-2.655756e-01,0.000104,0.196097
6,cep120,0.282444,0.004663,-0.283483,0.004086,0.177809,-6.230698e-03,0.257720,0.004528,0.286699,...,-0.264842,-0.005005,-0.294544,0.005464,2.485302e-01,0.003196,-0.207021,-6.436877e-03,0.271997,0.003050
7,dock1,0.279923,-0.000049,-0.282028,-0.000044,0.178907,7.751814e-05,0.260406,0.000169,0.289913,...,-0.268911,0.000073,-0.295004,0.000056,2.433755e-01,0.000462,-0.204942,-2.267067e-04,0.271775,-0.000280
8,hif1an,-0.000056,0.243359,0.000119,0.058190,-0.000126,-2.650217e-01,-0.000043,0.253729,-0.000252,...,0.000128,-0.224801,0.000142,0.271431,-9.885390e-05,0.237610,0.000054,-2.694988e-01,-0.000070,0.192423
9,rprd1a,0.000751,0.244381,-0.000102,0.061380,0.000739,-2.651821e-01,0.001146,0.254505,0.001039,...,-0.000881,-0.228986,-0.000826,0.270805,1.271500e-03,0.233022,-0.001057,-2.665453e-01,0.000837,0.195047
